<a href="https://colab.research.google.com/github/xloem/techsketball/blob/wip/model_import_sketch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
#[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googlecolab/colabtools/blob/master/notebooks/colab-github-demo.ipynb)

import jax

starting_model_path = 'baffo32/pyc2py_alpha' #'t5-base'#'t5-small'#'bigscience/T0pp'

input_width = 512
# these are not t5 parameters?
train_batch_size = 19 # small for notebook
per_device_batch_size = train_batch_size // jax.device_count()
num_epochs = 2
training_seed = 0
learning_rate = 3e-4
logging_steps = 1


#!pip install jax[tpu] -f https://storage.googleapis.com/jax-releases/libtpu_releases.html
!pip3 install deepspeed
!pip3 install transformers
!pip3 install flax
!pip3 install sentencepiece
!git clone https://github.com/xloem/techsketball && ln -s techsketball/* .

fatal: destination path 'techsketball' already exists and is not an empty directory.


In [12]:
import jax.tools.colab_tpu
import jaxlib
import os
try:
  if 'COLAB_TPU_ADDR' in os.environ:
    jax.tools.colab_tpu.setup_tpu()
  jaxlib.xla_extension.tpu_client()
  backend = 'tpu'
except:
  try:
    jaxlib.xla_extension.gpu_client()
    backend = 'gpu'
  except:
    backend = 'cpu'
import tensorflow as tf
!nvidia-smi
jax.local_devices()

Tue Jan 18 03:02:40 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    34W / 250W |  14799MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

[GpuDevice(id=0, process_index=0)]

In [13]:
from transformers import T5Tokenizer, FlaxT5ForConditionalGeneration 
import huggingface_hub
#repo = huggingface_hub.Repository('local_model', clone_from='hub_model_id')
try:
    tokenizer = T5Tokenizer.from_pretrained('local_model')
except:
    tokenizer = T5Tokenizer.from_pretrained(starting_model_path)
try:
    model = FlaxT5ForConditionalGeneration.from_pretrained('local_model')
except:
    model = FlaxT5ForConditionalGeneration.from_pretrained(starting_model_path)

In [14]:
# before data is generated we can import libraries to generate it from
import jax, jax.numpy as jnp
import numpy as np
import optax
import flax
import flax.training.common_utils
import flax.training.train_state
import tqdm
import time
import os
# ...
import transformers
import scipy

In [15]:
import find_pycode
print('getting training data ...')
tokenizerpfx = starting_model_path.replace('/','_') + '.'
find_pycode.write_files('example.', tokenizerpfx, 512, tokenizer, 512, globals(), skip_if_exists = True, train_tokenizer = True, tokenize_binary = True)
tokenizer.save_pretrained('local_model')
# repo.push_to_hub(commit_message=f'commit-message', blocking=False)
train_data = find_pycode.read_files('example.', tokenizerpfx, 512, 512, tokenize_binary = True)

getting training data ...


/usr/local/lib/python3.7/dist-packages/torch/distributed/distributed_c10d.py:171: UserWarning: torch.distributed.reduce_op is deprecated, please use torch.distributed.ReduceOp instead
  "torch.distributed.reduce_op is deprecated, please use "


In [16]:
#from tokenizers import ByteLevelBPETokenizer
#tokenizer = ByteLevelBPETokenizer()
#tokenizer.train_from_iterator((str for bytes, str in data_tuples), vocab_size=model.config.vocab_size, min_frequency=2) 

In [17]:
#import deepspeed
#cmd_args = None
#model_engine, optimizer, _, _ = deepspeed.initialize(args=cmd_args,
#                                                     model=model,
#                                                     model_parameters=params)

In [18]:
num_train_steps = len(train_data['input_ids']) // train_batch_size * num_epochs

rng = jax.random.PRNGKey(training_seed)

In [19]:
def batch_from_indices(dataset : dict, indices):
  #print(dataset['input_ids'].shape, indices.shape)
  result = {
      key : jnp.stack(data[indices,:])
      for key, data in dataset.items()
  }
  # this change could be already put in the dataset passed by the function that produces it
  result['labels'] = result['decoder_input_ids']
  result['decoder_input_ids'] = np.asarray(transformers.models.t5.modeling_flax_t5.shift_tokens_right(result['labels'], tokenizer.pad_token_id, model.config.decoder_start_token_id))
  return result

In [20]:
# these are not t5 parameters?
linear_decay_lr_schedule_fn = optax.linear_schedule(init_value=learning_rate, end_value=0, transition_steps=num_train_steps)
adamw = optax.adamw(learning_rate=linear_decay_lr_schedule_fn, b1=0.9, b2=0.98, eps=1e-8, weight_decay=0.01)
state = flax.training.train_state.TrainState.create(apply_fn=model.__call__, params=model.params, tx=adamw)

jax.config.update('jax_log_compiles', True)


# from run_t5_mlm_flax.py
dropout_rngs = jax.random.split(rng, jax.local_device_count())

# Define gradient update step fn
def train_step(state, batch, dropout_rng):#input_ids, attention_mask, labels, decoder_input_ids, decoder_attention_mask, dropout_rng):
    dropout_rng, new_dropout_rng = jax.random.split(dropout_rng)
    def loss_fn(params):
        labels = batch.pop('labels')

        #logits = state.apply_fn(
        #    input_ids = input_ids,
        #    attention_mask = attention_mask,
        #    decoder_input_ids = decoder_input_ids,
        #    decoder_attention_mask = decoder_attention_mask,
        #    params = params,
        #    dropout_rng = dropout_rng,
        #    train = True
        #).logits
        logits = state.apply_fn(**batch, params = params, dropout_rng = dropout_rng, train = True).logits
        #print(logits.shape)
        #assert len(logits[-1]) == tokenizer.vocab_size
        #logits = logits[0]

        # logits, labels, padding_mask=batch['decoder_attention_mask', label_smoothing_factor=0]
        # compute loss
        loss = optax.softmax_cross_entropy(logits, flax.training.common_utils.onehot(labels, logits.shape[-1]))
        padding_mask = batch['decoder_attention_mask']
        loss = (loss * padding_mask).sum() / padding_mask.sum()

        #loss = state.apply_fn(**batch, params=params, dropout_rng=dropout_rng, train=True).loss

        return loss

    grad_fn = jax.value_and_grad(loss_fn)
    loss, grad = grad_fn(state.params)
    grad = jax.lax.pmean(grad, "batch")
    new_state = state.apply_gradients(grads=grad)

    metrics = jax.lax.pmean(
        {"loss": loss, "learning_rate": linear_decay_lr_schedule_fn(state.step)}, axis_name="batch"
    )

    return new_state, metrics, new_dropout_rng

# Create parallel version of the train step
p_train_step = jax.pmap(train_step, 'batch', donate_argnums=(0,), backend=backend)

# Replicate the train state on each device
state = flax.jax_utils.replicate(state)

print('Performing initial batch to compile train step ...')
rng, input_rng = jax.random.split(rng)
num_train_samples = len(train_data['input_ids'])
train_samples_idx = jax.random.permutation(input_rng, jnp.arange(num_train_samples))
model_inputs = batch_from_indices(train_data, train_samples_idx[:train_batch_size])
model_inputs = flax.training.common_utils.shard(model_inputs)
state, train_metric, dropout_rngs = p_train_step(state, model_inputs, dropout_rng=dropout_rngs)
train_metric = flax.jax_utils.unreplicate(train_metric)
print('Done.  First loss was', train_metric['loss'].mean())

Performing initial batch to compile train step ...


Done.  First loss was 6.166926


In [ ]:


train_time = 0
epochs = tqdm.tqdm(range(num_epochs), desc="Epoch ... ", position=0)
for epoch in epochs:
    # ======================== Training ================================
    train_start = time.time()
    train_metrics = []

    # Create sampling rng
    rng, input_rng = jax.random.split(rng)

    # Generate an epoch by shuffling sampling indices from the train dataset
    num_train_samples = len(train_data['input_ids'])
    train_samples_idx = jax.random.permutation(input_rng, jnp.arange(num_train_samples))
    #train_batch_idx = generate_batch_splits(train_samples_idx, train_batch_size)

    # Gather the indexes for creating the batch and do a training step
    batches_tqdm = tqdm.tqdm(range(num_train_samples // train_batch_size), desc="Training...", position=1)
    for step, batch_idx in enumerate(batches_tqdm):
        #samples = [tokenized_datasets["train"][int(idx)] for idx in batch_idx]
        model_inputs = batch_from_indices(train_data, train_samples_idx[train_batch_size * batch_idx:train_batch_size * batch_idx + train_batch_size])
        #print('model_inputs are', {key:val.shape for key, val in model_inputs.items()})
        #model_inputs = data_collator(samples)

        # Model forward
        model_inputs = flax.training.common_utils.shard(model_inputs)
        state, train_metric, dropout_rngs = p_train_step(state, model_inputs, dropout_rng=dropout_rngs)
        train_metrics.append(train_metric)

        cur_step = epoch * (num_train_samples // train_batch_size) + step

        if cur_step % logging_steps == 0 and cur_step > 0 and jax.process_index() == 0:
            # Save metrics
            train_metric = flax.jax_utils.unreplicate(train_metric)
            train_time += time.time() - train_start
            #if has_tensorboard and jax.process_index() == 0:
            #    write_train_metric(summary_writer, train_metrics, train_time, cur_step)

            epochs.write(
                f"Loss: {train_metric['loss'].mean()}, Learning Rate: {train_metric['learning_rate'].mean()}"
            )

            train_metrics = []
        #if cur_step % (num_train_samples // 8) == 0:
            # save checkpoint
            if jax.process_index() == 0:
                params = jax.device_get(jax.tree_map(lambda x: x[0], state.params))
                model.save_pretrained('local_model', params=params)
                # repo.push_to_hub(commit_message=f'commit-message', blocking=False)

Training...:   0%|          | 0/1178 [00:00<?, ?it/s]

Training...:   0%|          | 1/1178 [02:51<27:40:53, 84.67s/it]

Loss: 4.415671348571777, Learning Rate: 0.00029974535573273897




Training...:   0%|          | 2/1178 [04:21<28:39:30, 87.73s/it]

Loss: 3.754929542541504, Learning Rate: 0.0002996179973706603




Training...:   0%|          | 3/1178 [05:53<29:25:41, 90.16s/it]

...



Training...:  12%|█▏        | 137/1178 [3:24:33<26:48:59, 92.74s/it]

Loss: 1.623967170715332, Learning Rate: 0.000282427849015221




Training...:  12%|█▏        | 138/1178 [3:26:02<26:29:52, 91.72s/it]

Loss: 2.1767661571502686, Learning Rate: 0.0002823005197569728



Training...:  12%|█▏        | 139/1178 [3:26:10<26:35:02, 92.11s/it]

In [ ]:
while True:
  print(repr(eval(input('>>> '), globals(), locals())))